In [14]:
from translation_utils import translate_column
import pandas as pd
from datetime import datetime

## Google Translation

In [16]:
# Import allGoogleReviews_2025-04-05
allGoogleReviews_2025_04_06 = pd.read_excel("reviews/google/allGoogleReviews_2025-04-06.xlsx")

# Filter out where originalLanguage is "en"
filtered_df = allGoogleReviews_2025_04_06[allGoogleReviews_2025_04_06['originalLanguage'] != "en"]

# Filter out where responseFromOwnerText is shorter than 5 characters
filtered_df = filtered_df[filtered_df['responseFromOwnerText'].str.len() > 5]

# Filter out where responseFromOwnerText_en is not null
filtered_df = filtered_df[filtered_df['responseFromOwnerText_en'].isnull()]

# Count the total number of characters in responseFromOwnerText
total_characters = filtered_df['responseFromOwnerText'].str.len().sum()
print(f"Total characters in responseFromOwnerText: {total_characters}")


Total characters in responseFromOwnerText: 142


In [17]:
# If you want to limit the total characters to 450000, you need to select rows until you reach that limit
if total_characters > 450000:
    # Create a cumulative sum of character lengths
    filtered_df['cumulative_chars'] = filtered_df['responseFromOwnerText'].str.len().cumsum()
    # Filter to keep only rows where cumulative characters are less than 450000
    filtered_df = filtered_df[filtered_df['cumulative_chars'] <= 450000]
    # Remove the temporary column
    filtered_df = filtered_df.drop('cumulative_chars', axis=1)

# First count the total characters
total_characters = filtered_df['responseFromOwnerText'].str.len().sum()
print(f"Total characters in responseFromOwnerText: {total_characters}")


Total characters in responseFromOwnerText: 142


In [18]:
# Find the difference between allGoogleReviews_2025-04-05 and filtered_df based on reviewId and save them to a new pd dataframe
difference_df = allGoogleReviews_2025_04_06[~allGoogleReviews_2025_04_06['reviewId'].isin(filtered_df['reviewId'])]

In [19]:
# Get the translated DataFrame without saving
translated_df, chars_translated = translate_column(
    input_data=filtered_df,  # or file path
    source_column="responseFromOwnerText",
    target_column="responseFromOwnerText_en",  # optional
    translation_service="deepseek",  # "deepl" or "deepseek"
    output_path=None  # Don't save, just return the DataFrame
)

# Do additional processing on translated_df
# Combine the translated_df with the difference_df based on reviewId
combined_df = pd.concat([difference_df, translated_df])

# Save the processed DataFrame when you're ready
output_path = "C:/Users/yigit/Desktop/Enterprises/verisanus_2/a_dev/reviews/google/processed_reviews.xlsx"
combined_df.to_excel(output_path, index=False)

Translating 3 rows using DEEPSEEK...
Total characters translated: 142


## Trustpilot Translation

### replyMessage

In [ ]:
# replyMessage,	reviewBody,	reviewHeadline


In [ ]:
# Translate the replyMessage column in allTrustpilotReviews_2025-04-06 to English, using deepseek. Save the result to a new column called replyMessage_en
# The input file is located at reviews/trustpilot/allTrustpilotReviews_2025-04-06.xlsx. The output file should be saved as processed_trustpilot_reviews.xlsx in the same directory.
allTrustpilotReviews_2025_04_06 = pd.read_excel("reviews/trustpilot/allTrustpilotReviews_2025-04-06.xlsx")
# Filter out where originalLanguage is "en"
filtered_df = allTrustpilotReviews_2025_04_06[allTrustpilotReviews_2025_04_06['reviewLanguage'] != "en"]
# Filter out where replyMessage is shorter than 5 characters
filtered_df = filtered_df[filtered_df['replyMessage'].str.len() > 5]
# Filter out where replyMessage_en is not null
filtered_df = filtered_df[filtered_df['replyMessage_en'].isnull()]
# Count the total number of characters in replyMessage
total_characters = filtered_df['replyMessage'].str.len().sum()
print(f"Total characters in replyMessage: {total_characters}")



Total characters in replyMessage: 24246


In [4]:
# Find the difference between allTrustpilotReviews_2025-04-06 and filtered_df based on reviewId and save them to a new pd dataframe
difference_df = allTrustpilotReviews_2025_04_06[~allTrustpilotReviews_2025_04_06['reviewId'].isin(filtered_df['reviewId'])]
# Get the translated DataFrame without saving
translated_df, chars_translated = translate_column(
    input_data=filtered_df,  # or file path
    source_column="replyMessage",
    target_column="replyMessage_en",  # optional
    translation_service="deepseek",  # "deepl" or "deepseek"
    output_path=None  # Don't save, just return the DataFrame
)
# Do additional processing on translated_df
# Combine the translated_df with the difference_df based on reviewId
combined_df = pd.concat([difference_df, translated_df])

Translating 101 rows using DEEPSEEK...


c:\Users\yigit\Desktop\Enterprises\verisanus_2\a_dev\translation_utils.py:164: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Dear Lorenzo, thank you so much for your review! 🙏🏻🙏🏻🙏🏻🤩🤩🤩' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[idx, target_column] = translated_text


Total characters translated: 24246


In [7]:
# Save the processed DataFrame when you're ready
output_path = "C:/Users/yigit/Desktop/Enterprises/verisanus_2/a_dev/reviews/trustpilot/allTrustpilotReviews_2025-04-06.xlsx"
combined_df.to_excel(output_path, index=False)

### reviewBody

In [8]:
# Translate the reviewBody column in allTrustpilotReviews_2025-04-06 to English, using deepseek. Save the result to a new column called reviewBody
# The input file is located at reviews/trustpilot/allTrustpilotReviews_2025-04-06.xlsx. The output file should be saved as processed_trustpilot_reviews.xlsx in the same directory.
allTrustpilotReviews_2025_04_06 = pd.read_excel("reviews/trustpilot/allTrustpilotReviews_2025-04-06.xlsx")
# Filter out where originalLanguage is "en"
filtered_df = allTrustpilotReviews_2025_04_06[allTrustpilotReviews_2025_04_06['reviewLanguage'] != "en"]
# Filter out where reviewBody is shorter than 5 characters
filtered_df = filtered_df[filtered_df['reviewBody'].str.len() > 5]
# Filter out where reviewBody_en is not null
filtered_df = filtered_df[filtered_df['reviewBody_en'].isnull()]
# Count the total number of characters in reviewBody
total_characters = filtered_df['reviewBody'].str.len().sum()
print(f"Total characters in reviewBody: {total_characters}")



Total characters in reviewBody: 148933


In [9]:
# Find the difference between allTrustpilotReviews_2025-04-06 and filtered_df based on reviewId and save them to a new pd dataframe
difference_df = allTrustpilotReviews_2025_04_06[~allTrustpilotReviews_2025_04_06['reviewId'].isin(filtered_df['reviewId'])]
# Get the translated DataFrame without saving
translated_df, chars_translated = translate_column(
    input_data=filtered_df,  # or file path
    source_column="reviewBody",
    target_column="reviewBody_en",  # optional
    translation_service="deepseek",  # "deepl" or "deepseek"
    output_path=None  # Don't save, just return the DataFrame
)
# Do additional processing on translated_df
# Combine the translated_df with the difference_df based on reviewId
combined_df = pd.concat([difference_df, translated_df])

Translating 295 rows using DEEPSEEK...


c:\Users\yigit\Desktop\Enterprises\verisanus_2\a_dev\translation_utils.py:164: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'I underwent the transplant at Hair of Istanbul in April 2023. They picked me up with a private driver and the very kind Chala, the clinic's interpreter and assistant, who never left me alone for a moment. After checking the donor area, the procedure began and lasted about 8 hours. Extremely professional. Of course, at the moment, I cannot judge the level of results because I had the surgery recently, and it will take a few months to see the actual outcome. However, I can say that I was followed from the first to the last moment. Even in the days following the surgery, as soon as I returned, I interacted with the clinic and always found them extremely available.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[idx, target_column] = translated_t

Total characters translated: 148933


In [10]:
# Save the processed DataFrame when you're ready
output_path = "C:/Users/yigit/Desktop/Enterprises/verisanus_2/a_dev/reviews/trustpilot/allTrustpilotReviews_2025-04-06.xlsx"
combined_df.to_excel(output_path, index=False)

### reviewHeadline

In [11]:
# Translate the reviewHeadline column in allTrustpilotReviews_2025-04-06 to English, using deepseek. Save the result to a new column called replyMessage_en
# The input file is located at reviews/trustpilot/allTrustpilotReviews_2025-04-06.xlsx. The output file should be saved as processed_trustpilot_reviews.xlsx in the same directory.
allTrustpilotReviews_2025_04_06 = pd.read_excel("reviews/trustpilot/allTrustpilotReviews_2025-04-06.xlsx")
# Filter out where originalLanguage is "en"
filtered_df = allTrustpilotReviews_2025_04_06[allTrustpilotReviews_2025_04_06['reviewLanguage'] != "en"]
# Filter out where reviewHeadline is shorter than 5 characters
filtered_df = filtered_df[filtered_df['reviewHeadline'].str.len() > 5]
# Filter out where reviewHeadline is not null
filtered_df = filtered_df[filtered_df['reviewHeadline_en'].isnull()]
# Count the total number of characters in reviewHeadline
total_characters = filtered_df['reviewHeadline'].str.len().sum()
print(f"Total characters in reviewHeadline: {total_characters}")



Total characters in reviewHeadline: 8446


In [12]:
# Find the difference between allTrustpilotReviews_2025-04-06 and filtered_df based on reviewId and save them to a new pd dataframe
difference_df = allTrustpilotReviews_2025_04_06[~allTrustpilotReviews_2025_04_06['reviewId'].isin(filtered_df['reviewId'])]
# Get the translated DataFrame without saving
translated_df, chars_translated = translate_column(
    input_data=filtered_df,  # or file path
    source_column="reviewHeadline",
    target_column="reviewHeadline_en",  # optional
    translation_service="deepseek",  # "deepl" or "deepseek"
    output_path=None  # Don't save, just return the DataFrame
)
# Do additional processing on translated_df
# Combine the translated_df with the difference_df based on reviewId
combined_df = pd.concat([difference_df, translated_df])

Translating 293 rows using DEEPSEEK...


c:\Users\yigit\Desktop\Enterprises\verisanus_2\a_dev\translation_utils.py:164: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Hair of Istanbul …' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[idx, target_column] = translated_text


Total characters translated: 8446


In [13]:
# Save the processed DataFrame when you're ready
output_path = "C:/Users/yigit/Desktop/Enterprises/verisanus_2/a_dev/reviews/trustpilot/allTrustpilotReviews_2025-04-06.xlsx"
combined_df.to_excel(output_path, index=False)